# Tom O'Leary-Roseberry
## Homework 4

### 2 Programming

We are interested in solving:

$\min\limits_x \sum\limits_{i=1}^m \|A_ix_i - b_i\|^2 + \lambda \sum\limits_{i=1}^m \sum\limits_{j=i+1}^m \|x_i - x_j\|_1$

Using proximal gradient methods using some proximal gradient method and generated data set.

Let $A_i \in \mathbb{R}^{n\times k}$ and $b_i \in \mathbb{R}^n$, so $x_i \in \mathbb{R}^k$ 

We choose to think of $\{x_i\}_{i=1}^m = X$ as a data matrix $X \in \mathbb{R}^{m \times k}$ based on how it imports from the matlab matrix file.


At a given iteration of proximal gradients, we compute the gradient with respect to the $C^1$ portion of the objective function

$\sum\limits_{i=1}^m \|A_ix_i - b_i\|^2$

and then filter with respect to the $C^0$ sparsifying penalty term 

$\sum\limits_{i=1}^m \sum\limits_{j=i+1}^m \|x_i - x_j\|_1$

The filtering process amounts to the derivation of the proximity operator, which for a function $\phi$ is defined as follows:

$\text{prox}_\phi = \text{arg}\min\limits_{x \in \mathcal{H}} \phi(x) + \frac{1}{2}\|u - x\|^2_2$

The algorithm for calculating the proximity operator is defined as follows for this problem:

1. 
    a. At a given iteration each row of $X$ is sorted $(x_0,x_1,\dots, x_m)$ such that $x_0 \leq x_1 \leq \dots \leq x_m$
    
    b. In order to account for degenerate cases where two coordinates are equal the list is collapsed into a list of tuples:
    
    $\{ (x_i,l_i) | l_i = \#\{x_i \text{in} (x_0,x_1,\dots, x_m)\} \}$
    
    let the length of this list be $n$
    
2. The problem can now be posed as the following minimization problem

    $u = \text{arg}\min\limits_{u_i}  \sum\limits_{i=1}^n \sum\limits_{j=i+1}^n \lambda l_i l_j(u_i - u_j) + \sum\limits_{i=1}^n \frac{l_i}{2} |u_i - x_i|^2$

    Note that an immediate consequence is the property that if $u$ is a minimizer of this problem then $u_0 \leq u_1 \leq\dots $

    If this were not the case at some point in the list the ordering was not monotonic then there would be one term in the first term of the objective function that would increase the cost relative to the ordering being flipped. This is a contraction and the list must be monotonic.

3. We can then pose the equivalent saddle point problem with Lagrange multiplier $\mu$ that penalizes deviations from this optimal property

    $\min\limits_{u_i}\max\limits_{\mu_i\geq 0} \mathcal{L}(u,\mu) =\min\limits_{u_i}\max\limits_{\mu_i\geq 0}\sum_k \mu_k(u_k - u_{k+1}) + \sum\limits_{i<j}l_il_j(u_j - u_i) + \sum\limits_i \frac{l_i}{2}|u_i - x_i|^2 $

    Where the counter $l$ allows us to collapse entire sums by counting up the number of redudant terms with $l_i l_j$ etc.
    
4. Next define the coefficient:

    $ c_k = \lambda \bigg(\sum_{j>k} l_j - \sum_{k>j}l_j \bigg) + x_k$
    
    The stationary points of the Lagrangian can now be described algebraically:

    $ \frac{\partial \mathcal{L}}{\partial u_k} = \mu_k - \mu_{k-1} - l_k c_k + l_k u_k = 0$

    Observe that if $c_k \geq c_{k+1}$ then $u_k = u_{k+1}$. To see this suppose otherwise and assume $u_k < u_{k+1}$ since the other ordering is not optimal. then this implies that
    
    $c_k + \frac{1}{l_k}\bigg(\mu_{k-1} - \mu_k \bigg) < c_{k+1} + \frac{1}{l_{k+1}}\bigg(\mu_{k} - \mu_{k+1} \bigg)$
    
    which implies that $\mu_k>0$ but we know from the stationary condition for the Lagrangian that $\mu_k(u_k -u_{k+1}) = 0$ which then gives that either the ordering was assumed in err or $\mu_k = 0$. Either case contradiction yields the result.
       
5. So now the coordinates of each row of $X - \alpha \nabla F(X)$ may be sorted at each iteration, and the above terms may be evaluated, if $c_k \geq c_{k+1}$ then we must merge the tuples $\{(x_k,l_k),(x_{k+1},l_{k+1})\}$. When we arrive at an ordering such that $c_1 < c_2<\dots $ then we know that we have found a minimum of the proximity operator minimization problem. This list is then the updated control at a given Armijo backtracking iteration.
    

In [1]:
import numpy as np
import numpy.linalg as la
from scipy.io import loadmat

Data = loadmat('hw4_data.mat')
A_temp = Data['A']
b_temp = Data['b']
m = len(A_temp)
n, k =  A_temp[0][0].shape

A = []
b = []
for i in range(len(A_temp)):
    A.append(A_temp[i][0])
    b.append(b_temp[i][0])
    
A = np.array(A)
b = np.array(b)

print (A.shape, b.shape)

((10, 300, 1000), (10, 300, 1))


In [2]:
class sparse_vector_recovery:
    def __init__(self,A,b,lmbda=1e-4):
        self.A = A
        self.b = b
        self.m, self.n,self.k = self.A.shape
        self.x = np.zeros((self.m,self.k,1))
        self.p = None
        self.lmbda = lmbda
    def evaluate(self, x = None):
        if x is None:
            x = self.x
        cost = 0.0
        for i in range(self.m):
            Ax_bi = np.dot(A[i],x[i]) - b[i]
            cost += np.vdot(Ax_bi,Ax_bi)
        return cost
    
    def weightedsum(self,xklk,xjlj):
        (xk,lk) = xklk
        (xj,lj) = xjlj
        return [(lk*xk+lj*xj)/(lk+lj),(lk+lj)]

    def C(self,x_l,k):
        assert(k<len(x_l))
        x = np.array([x_temp for [x_temp,l_temp] in x_l])
        l = np.array([l_temp for [x_temp,l_temp] in x_l])
        Ck = x[k]
        Ck+=self.lmbda*(sum(l[k+1:])-sum(l[0:k]))
        return Ck

    def prox(self,z):
        for i in range(z.shape[1]):
            x =  np.copy(z[:,i,0])
            
            #sort and store original indices
            sortedx=np.sort(x)
            originalxmapping=np.argsort(x)

            # get unique x's and counts
            uniquexs,counts = np.unique(x, return_counts=True)
            x_l = [[x,y] for (x,y) in zip(uniquexs,counts)]
        
            #contract
            contractedX_l = []
            k = 0
            while k < len(x_l)-1:
                if (self.C(x_l,k)>=self.C(x_l,k+1)):
                    x_l[k+1] = self.weightedsum(x_l[k],x_l[k+1])
                    if ((k+1)==len(x_l)-1):
                        contractedX_l.append(x_l[k+1])
                else:
                    contractedX_l.append(x_l[k])
                    if ((k+1)==len(x_l)-1):
                        contractedX_l.append(x_l[k+1])
                k+=1

            #[(C_i,l) for i in ..]
            proximalXsorted = [[self.C(contractedX_l,i),contractedX_l[i][1]] for i in range(len(contractedX_l))]

            #flatten
            flatproximalXsorted = []
            for (proximalX,num) in proximalXsorted:
                for i in range(num):
                    flatproximalXsorted.append(proximalX)

            proximalX = np.array([x for _,x in sorted(zip(originalxmapping,flatproximalXsorted))])

            #store proximal X in the data array
            z[:,i,0]=proximalX
        return z

    def gradient(self,x = None):
        if x is None:
            x = self.x
        grad = np.zeros_like(x)
        for i in range(self.m):
            Ax_bi = np.dot(A[i],x[i]) - b[i]
            grad[i] += 2*np.dot(self.A[i].T,Ax_bi)
        return grad
    
    def grad_norm(self,x = None):
        if x is None:
            x = self.x
        return la.norm(self.gradient(x))
    
    
handler = sparse_vector_recovery(A,b) 

handler.grad_norm()
        

16298.048288201997

In [3]:
def proximal_steepest_descent(handler,A,b,x_0, tolerance = 1e-8,
             max_iters = 10000,back_track = False, eta_option =0,
             min_step = 1e-10,c_armijo = 1e-4,verbose = False):
    f = handler(A,b)
    f.p = np.zeros_like(x_0)
    iters = 0
    alpha = 1.0
    grad_norm0 = f.grad_norm()
    grad_norm = grad_norm0
    
    while grad_norm > tolerance*grad_norm0 and iters <= max_iters:
        cost = f.evaluate()
        g = f.gradient()
        f.p = -g
        p_inner_g = np.vdot(g,f.p)
        if back_track:
            cost_old = cost
            alpha = 1.
            while alpha > min_step:
                temp = f.prox(f.x + alpha*f.p)
                cost = f.evaluate(temp)
                if cost < cost_old -c_armijo*alpha*p_inner_g:
                    break
                else:
                    alpha *= 0.5
        f.x = f.prox(f.x+alpha*f.p)
        grad_norm = f.grad_norm()
        if verbose:
            if iters == 0:
                print ("\n{0:10} {1:15} {2:15} {3:15}".format(
                      "Iteration",  "cost", "||g||L2", "alpha" ))
            else:
                print ("\n{0:<10d} {1:<15e} {2:<15e} {3:<15e}".format(
                      iters,  cost, grad_norm, alpha ))
        iters += 1
        converged = (grad_norm < tolerance*grad_norm0)
    return f.x,iters, converged

x_0 = np.ones((m,k,1))

x,iters,converged = proximal_steepest_descent(sparse_vector_recovery,A,b,x_0,\
                                     back_track=True, verbose = True)

print x


Iteration  cost            ||g||L2         alpha          

1          6.599190e+04    6.427157e+03    3.906250e-03   

2          1.493264e+04    2.774220e+03    3.906250e-03   

3          5.139530e+03    1.355570e+03    3.906250e-03   

4          2.398500e+03    7.429198e+02    3.906250e-03   

5          1.331183e+03    4.599147e+02    3.906250e-03   

6          8.821832e+02    6.162923e+02    7.812500e-03   

7          4.080506e+02    3.406987e+02    3.906250e-03   

8          3.558171e+02    2.907067e+02    4.882812e-04   

9          2.082665e+02    1.819972e+02    3.906250e-03   

10         1.619256e+02    2.694211e+02    7.812500e-03   

11         7.701721e+01    1.520726e+02    3.906250e-03   

12         4.611307e+01    7.025228e+01    1.953125e-03   

13         3.131216e+01    5.443445e+01    3.906250e-03   

14         1.322341e+01    8.796281e+01    1.562500e-02   

15         3.715043e+00    2.564158e+01    1.953125e-03   

16         2.436413e+00    1.716729e+01